### **Library Imports**

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, \
                            roc_auc_score, \
                            precision_recall_fscore_support, \
                            classification_report, \
                            confusion_matrix

In [2]:
class CFG(object):
    def __init__(self,
                 seed: int = 42,
                 n_splits: int = 5,
                 show_info: bool = False,
                 ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.data_read_path = "../input/cardiovascular-disease-dataset/cardio_train.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

### **Utilities and Helpers**

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")
    

def print_scores(accuracy: float, auc: float, precision: np.ndarray, recall: np.ndarray, f_score: np.ndarray) -> None:
    print(f"Accuracy  : {accuracy:.5f}")
    print(f"ROC-AUC   : {auc:.5f}")
    print(f"Precision : {precision}")
    print(f"Recall    : {recall}")
    print(f"F-Score   : {f_score}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    accuracy = accuracy_score(y_pred, y_true)
    auc = roc_auc_score(y_pred, y_true)
    precision, recall, f_score, _ = precision_recall_fscore_support(y_pred, y_true)

    return accuracy, auc, precision, recall, f_score

In [4]:
class Pipelines(object):
    def __init__(self, preprocessor, seed: int):
        self.model = Pipeline(
            steps=[
                ("preprocessor", preprocessor),
                ("classifier", AdaBoostClassifier(random_state=seed)),
            ]
        )

### **Normal**

In [5]:
df = pd.read_csv(cfg.data_read_path, delimiter=";")
df = df.drop(columns=["id"])
df.age /= 365.25

X = df.iloc[:, :-1].copy().values
y = df.iloc[:, -1].copy().values

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Simple_Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

In [6]:
best_auc = 0.0
best_acc = 0.0
best_pre = 0.0
best_rec = 0.0
best_f1  = 0.0

fold = 1

breaker()
for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
    X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
    my_pipeline = Pipelines(preprocessor, cfg.seed)
    my_pipeline.model.fit(X_train, y_train)

    y_pred = my_pipeline.model.predict(X_valid)
    acc, auc, pre, rec, f1 = get_scores(y_valid, y_pred)
    print(f"Fold {fold}\n")
    print_scores(acc, auc, pre, rec, f1)
    print("")
    print(confusion_matrix(y_valid, y_pred))
    breaker()

    if acc > best_acc:
        best_acc = acc
        best_acc_fold = fold
    
    if auc > best_auc:
        best_auc = auc
        best_auc_fold = fold

    if pre[1] > best_pre:
        best_pre = pre[1]
        best_pre_fold = fold
    
    if rec[1] > best_rec:
        best_rec = rec[1]
        best_rec_fold = fold
    
    if f1[1] > best_f1:
        best_f1 = f1[1]
        best_f1_fold = fold
    
    fold += 1

print(f"Best ACC Fold : {best_acc_fold}")
print(f"Best AUC Fold : {best_auc_fold}")
print(f"Best PRE Fold : {best_pre_fold}")
print(f"Best REC Fold : {best_rec_fold}")
print(f"Best F1 Fold  : {best_f1_fold}")

breaker()


**************************************************

Fold 1

Accuracy  : 0.73536
ROC-AUC   : 0.73994
Precision : [0.80409273 0.66685682]
Recall    : [0.70634821 0.77353184]
F-Score   : [0.75205782 0.71624416]

[[5619 1369]
 [2336 4676]]

**************************************************

Fold 2

Accuracy  : 0.73200
ROC-AUC   : 0.73642
Precision : [0.79890679 0.66600454]
Recall    : [0.70232676 0.77051871]
F-Score   : [0.74751009 0.71445967]

[[5554 1398]
 [2354 4694]]

**************************************************

Fold 3

Accuracy  : 0.72964
ROC-AUC   : 0.73325
Precision : [0.7906075  0.66931931]
Recall    : [0.7028856  0.76361868]
F-Score   : [0.74417033 0.71336615]

[[5505 1458]
 [2327 4710]]

**************************************************

Fold 4

Accuracy  : 0.72693
ROC-AUC   : 0.73063
Precision : [0.79496908 0.656595  ]
Recall    : [0.70527366 0.75597926]
F-Score   : [0.74744005 0.70279095]

[[5657 1459]
 [2364 4520]]

**************************************************


### **Grid Search**

In [7]:
param_grid = {
    "n_estimators" : (50, 100, 150, 200),
    "learning_rate" : (1, 1e-1, 1e-2, 1e-3),
}

sc_X = StandardScaler()
si_mean = SimpleImputer(missing_values=np.nan, strategy="mean")

X = si_mean.fit_transform(X)
X = sc_X.fit_transform(X)

my_pipeline = Pipelines(preprocessor, cfg.seed)

breaker()
gscv_model = GridSearchCV(
    estimator=AdaBoostClassifier(),
    param_grid=param_grid,
    cv=cfg.n_splits,
    verbose=3,
    return_train_score=True,
    ).fit(X, y)
breaker()


**************************************************

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END learning_rate=1, n_estimators=50;, score=(train=0.732, test=0.728) total time=   1.8s
[CV 2/5] END learning_rate=1, n_estimators=50;, score=(train=0.731, test=0.734) total time=   1.8s
[CV 3/5] END learning_rate=1, n_estimators=50;, score=(train=0.732, test=0.733) total time=   1.7s
[CV 4/5] END learning_rate=1, n_estimators=50;, score=(train=0.733, test=0.730) total time=   1.8s
[CV 5/5] END learning_rate=1, n_estimators=50;, score=(train=0.731, test=0.726) total time=   1.7s
[CV 1/5] END learning_rate=1, n_estimators=100;, score=(train=0.732, test=0.729) total time=   3.5s
[CV 2/5] END learning_rate=1, n_estimators=100;, score=(train=0.731, test=0.736) total time=   3.5s
[CV 3/5] END learning_rate=1, n_estimators=100;, score=(train=0.732, test=0.732) total time=   3.5s
[CV 4/5] END learning_rate=1, n_estimators=100;, score=(train=0.733, test=0.730) total time

In [8]:
breaker()
print(f"Best Estimator : {gscv_model.best_estimator_}")
print(f"Best Score     : {gscv_model.best_score_}")
print(f"Best Fold      : {gscv_model.best_index_}")
breaker()
print("Best Parameters\n")
for k, v in gscv_model.best_params_.items():
    # Left Justify the text making it 14 characte3rs long
    print(f"{k:<14} : {v}")
breaker()


**************************************************

Best Estimator : AdaBoostClassifier(learning_rate=1, n_estimators=150)
Best Score     : 0.7313428571428572
Best Fold      : 2

**************************************************

Best Parameters

learning_rate  : 1
n_estimators   : 150

**************************************************



### **Best Grid Seached Model**

In [9]:
df = pd.read_csv(cfg.data_read_path, delimiter=";")
df = df.drop(columns=["id"])
df.age /= 365.25

X = df.iloc[:, :-1].copy().values
y = df.iloc[:, -1].copy().values

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Simple_Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", AdaBoostClassifier(random_state=cfg.seed, n_estimators=150, learning_rate=1)),
    ]
)

breaker()
my_pipeline = Pipelines(preprocessor, cfg.seed)
my_pipeline.model.fit(X, y)

with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)


**************************************************

